This notebook is a summery of preprocessing and exploratory data analysis of two set of meRIP-Seq data from patient with COVID19. For each patient, we have sequence results for both input/IP for each sample. Here we use `exomePeak` for peak calling. To our experience, this is the only package comparing to other methods that supports samples from one conditions for the peak calling task; where other methods (like `RADAR`) require samples from two conditions to do both peak calling and differential analysis which is not our interest here.

Totally, there are 44 sequencing samples, 22 patients with both IN and RIP. 

Current pipeline contain these steps: 

#### Human genome 
0. Run `bcl2fastq` to make fastq files from raw data
        - input: bcl
        - output: fastq
1. Using `cutadapt` to trim few bps from the sequencing library.
        - input: fastq
        - output: fastq
2. Align to human genome using `STAR` and write unaligned reads in seprate `fastq` files
        - input: fastq
        - output1: bam
        - output2: fastq
3. Call human m6A peaks in each samples using `exomePeak` (see [here](https://bioconductor.riken.jp/packages/3.0/bioc/html/exomePeak.html)) and `gencode.v28.annotation`. 
        - input1: gtf
        - input2: bam
        - output: bed12

4. Taking bed12 results from step 3:
    1. Drawing meta-gene plot using `Guitar` (see [here](https://bioconductor.org/packages/3.11/bioc/html/Guitar.html)). 
    2. Using `cgat` (see [here](https://cgat.readthedocs.io/en/latest/cgat.html)) to prepare FIRE input. 
    2. Motif analysis using `FIRE`
    
Final results reported separately [here](https://github.com/goodarzilab/Abe/blob/master/Projects/COVID19_m6A/human-report.ipynb)

#### Virus genome 
5. Using `STAR` to build index and map COVID19 genome to unaligned reads to the human genome.
    According to Matt's epxeriance ([link](https://github.com/goodarzilab/khorms/blob/master/side_projects/COVID_DMSseq_library.ipynb)), I'm using [this genome annotation](https://www.ncbi.nlm.nih.gov/nuccore/MN908947?%3Fdb=nucleotide). 
        - input: fastq
            output2 of step 2. 
        - output: bam 


6. Using `exomePeak` with custom option that make it work for calling virus peaks. 
        - input1: gtf
        - input2: bam
        - output: bed12

7. Albertas is using m6A+ m6A- internal controls from [N6-Methyladenosine Enrichment Kit](https://international.neb.com/-/media/nebus/files/manuals/manuale1610.pdf?rev=c064d5e232414f709ef8bccee56f7687&hash=AAA94B8FDA043DD5A610E14686552B5E0209EA94). I'm using `bowtie2` to align reads in the `output2` from step 2. In a scatter plot, qPCR Ct for each patients is compered with number of virus m6A peaks.


8. Merge and intersect peaks found in all samples into a single `bed6` file and extract peak sequences. 
        - inputs: bed12
            outputs of step 6
        - output1: bed
        - output2: fasta

9. Count virus peaks as features using `bamToBed` plus `intersectBed` to count reads that aligned to each peaks. 
        - input1: bam 
            output of step 5.
        - input2: bed
            output1 of step 8. 
        - output: count-matrix 

        
10. Motif analysis using `regex` by extracting the sequence of each peaks and quary patterns
        - input: fasta
            output2 of step 8. 
        - output: motif-dataframe


12. Implement R code from [here](https://github.com/lzcyzm/exomePeak/blob/master/R/ctest.R) to run `ctest` which evaluate m6A vs. input foldchange, pvalue and fdr for every peaks in all 22 samples. 


11. Also, there are WGS `fasta` results for several samples. For mutation analysis, I'm applying fuzzy string matching which calculate **Levenshtein Distance** between each peak sequence within the sample sequence.

In [3]:
import re
import os 
import sys 
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz
from scipy import stats 
from matplotlib.patches import Rectangle
import sklearn.preprocessing as pp
import rpy2.robjects as ob

print (sys.version)

3.6.10 | packaged by conda-forge | (default, Apr 24 2020, 16:44:11) 
[GCC 7.3.0]


In [4]:
Samples = [
    "S0008","S0009","S0014","S0017","S0025",
    "S0026","S0030","S0042","S0057","S0085",
    "COV00075","COV00079","COV00084","COV00087","COV00093","COV00106",
    "COV00397","COV00413","COV00417","COV00419","COV00422","COV00432"    
]

In [5]:
renameSamples = [S.replace('S', 'COV0') for S in Samples]

(Some of below scripts repeated for both set of samples separately)

(I have used several separate conda environments for this analysis)

## 0. `bcl2fastq`

In [2]:
cat second_dataset/_sh/bcl2fastq.sh

cd ~/Projects/COVID19_m6A/second_dataset

mkdir -p fastq/
mkdir -p _sh/
mkdir -p _sh/bcl2fastq/

bcl2fastq --create-fastq-for-index-reads \
--runfolder-dir 200527_NS500257_0113_AH2VKLBGXF \
-r 18 -p 18 \
-o fastq/ \
--sample-sheet 200527_NS500257_0113_AH2VKLBGXF/200527_meRIPseq.csv \
--no-lane-splitting \
--stats-dir _sh/bcl2fastq/ \
--reports-dir _sh/bcl2fastq/


## 1. Trimming task

Removing the first three nucleotides of the second reads. 

In [14]:
cat second_dataset/_sh/trim.sh

mkdir -p trim
for fq in fastq/S00*R2*; do
	fq=`basename $fq`;
	out=${fq/_001.fastq.gz/.trim.fastq.gz};
	echo -------------------------$fq------------------------
	cutadapt -j 12 -u 3 -o trim/$out fastq/$fq
done


## 2. Alignment task

In [3]:
cat second_dataset/_sh/star.sh

cd ~/Projects/COVID19_m6A/second_dataset
mkdir -p bam

STAR --genomeLoad LoadAndExit --genomeDir /rumi/shams/genomes/hg38/
for fq in fastq/*R1*; do
    fq1=`basename $fq`
    fq2=${fq1/_R1_/_R2_}
    fq2=${fq2/.fastq.gz/.trim.fastq.gz}
    out=${fq1/_R1_001.fastq.gz/}
    STAR \
    --outSAMtype BAM SortedByCoordinate \
    --readFilesCommand zcat \
    --runThreadN 16 \
    --genomeDir /rumi/shams/genomes/hg38/ \
    --readFilesIn fastq/$fq1 trim/$fq2 \
    --outFileNamePrefix bam/$out \
    --outReadsUnmapped Fastx; 
done
STAR --genomeLoad Remove --genomeDir /rumi/shams/genomes/hg38/


## 3. Human peak calling 

In [4]:
cat second_dataset/_sh/peak.sh

MAIN=/rumi/shams/abe/Projects/COVID19_m6A/second_dataset
PEAKS=exomepeak/human

cd $MAIN
for f in bam/*_IN*.bam; do 
	b=`basename $f`; 
	b=${b/_IN.bam/}; 
	echo -------------------------$b------------------------
        Rscript _sh/exompeak.R $MAIN $PEAKS $b _IN _RIP
	echo 'All done!'
done


In [5]:
cat second_dataset/_sh/exompeak.R

library(GenomicFeatures)
library(exomePeak)

args <- commandArgs(trailingOnly = TRUE)

jobID  <- args[1]
OUTPUT <- args[2]
Sample <- args[3]
INPUT <- args[4]
IP <- args[5]

######################################## read meta ######################################$

IP_BAM = paste(Sample, IP, '.bam', sep='')
INPUT_BAM = paste(Sample, INPUT, '.bam', sep='')

print (IP_BAM)
print (INPUT_BAM)

######################################## functions ######################################
GTF = '/rumi/shams/genomes/hg38/gencode.v28.annotation.gtf'
organism= 'Homo sapiens'

txdb <- makeTxDbFromGFF(GTF, organism=organism )

setwd(jobID)
setwd("./bam")

print (txdb)

res <- exomepeak(
	TXDB=txdb,
	IP_BAM=IP_BAM,
	INPUT_BAM=INPUT_BAM,
	OUTPUT_DIR=paste('..',OUTPUT,sep='/'),
	EXPERIMENT_NAME=Sample
)
saveRDS(res, paste('..', OUTPUT, Sample, 'results.rds', sep='/'))


## 4. Downstream analysis of human data 

Here, I run `Guitar.R` to generate meta-gene plots and `motif.sh` for motif analysis of `DRACH` and `RGAC` motifs and also motif analysis in discovery mode. 

#### Draw meta-gene plots

In [6]:
cat second_dataset/_sh/guitar.sh

for f in bam/*_IN*.bam; do
	b=`basename $f`;
	b=${b/_IN.bam/};
	echo -------------------------$b------------------------;
	Rscript _sh/guitar.R exomepeak/human/$b peak.bed $b;
done


In [7]:
cat second_dataset/_sh/guitar.R

library(Guitar)

args <- commandArgs(trailingOnly = TRUE)

jobID <- args[1]
bed_file <- args[2]
plot_prefix <- args[3]

setwd(jobID)
list.files()
print (bed_file)

txdb <- makeTxDbFromGFF('/rumi/shams/genomes/hg38/gencode.v28.annotation.gtf',organism='Homo sapiens')

GuitarPlot(txTxdb=txdb,stBedFiles=list(bed_file),miscOutFilePrefix=plot_prefix)


#### Motif analysis 
Prepare inputs and run [FIRE](https://github.com/goodarzilab/FIRE).

In [8]:
cat second_dataset/_sh/motif.sh

MAIN=/rumi/shams/abe/Projects/COVID19_m6A/second_dataset
PEAKS=exomepeak/human
MOTIF=/rumi/shams/abe/Projects/COVID19_m6A/motifs.txt

cd ${MAIN}/${PEAKS}
for sam in *; do
	cd $sam
	# step 1: extract mRNA sequences
	cat peak.bed | sort -k1,1 -k2,2n peak.bed | cgat bed2bed --method=merge --merge-by-name |  awk '! /#/' | bedtools getfasta -name -s -fi /rumi/shams/genomes/hg38/hg38.fa -bed - -split -fo peak.fa
        # step 2: prepare inputs for FIRE
	perl $TEISERDIR/prep_seqs_for_teiser_run.pl peak.fa peaks
	# step 3: run FIRE for known m6A motifs (non-discovery mode)
	perl $FIREDIR/fire.pl --expfile=peaks_teiser.txt --exptype=discrete --fastafile_rna=peaks_teiser.fa \
	--nodups=1 --dodna=0 --dodnarna=0 --species=human --doskipdiscovery=1 --motiffile_rna=$MOTIF --oribiasonly=0
	mv -v peaks_teiser.txt_FIRE/ non-discovery_FIRE
	# step 4: run FIRE discovery mode
	perl $FIREDIR/fire.pl --expfile=peaks_teiser.txt --exptype=discrete --fastafile_rna=peaks_teiser.fa \
	--nodups=1 --dodna=0 --dod

# Virus genome

## 5. Alignment task for COVID19 genome

In [25]:
cat second_dataset/_sh/virus_star.sh

# map to virus genome 
# https://www.ncbi.nlm.nih.gov/nuccore/MN908947?%3Fdb=nucleotide
# STAR --runThreadN 16 \
# --runMode genomeGenerate \
# --genomeDir ../virus/ \
# --genomeFastaFiles ../virus/coronavirus_2_isolate_Wuhan-Hu-1.fasta \
# --sjdbGTFfile ../virus/coronavirus_2_isolate_Wuhan-Hu-1.gff3 \
# --sjdbOverhang 99 --genomeSAindexNbases 8 --sjdbGTFfeatureExon CDS

mkdir -p virus_bam
STAR --genomeLoad LoadAndExit --genomeDir ../virus/

for fq in virus_fastq/*mate1; do
	fq1=`basename $fq`;
	fq2=${fq1/mate1/mate2};
	out=${fq1/_Unmapped.out.mate1/};
	echo 'sample ' $fq1
	STAR \
	--outSAMtype BAM SortedByCoordinate \
	--readFilesCommand cat \
	--runThreadN 16 \
	--genomeDir ../virus/ \
	--readFilesIn virus_fastq/$fq1 virus_fastq/$fq2 \
	--outFileNamePrefix virus_bam/$out \
	--limitBAMsortRAM 1000000000;
done
STAR --genomeLoad Remove --genomeDir ../virus/




## 6. Virus peak calling 

I've played with `exomePeak` thresholds and we ended up using  ....

In [29]:
cat _sh/virus_peak.sh  

MAIN=/rumi/shams/abe/Projects/COVID19_m6A/
PEAKS=exomepeak/virus

cd $MAIN

for f in */bam/*_IN*.bam; do
	b=`basename $f`;
	b=${b/_IN.bam/};
	echo -------------------------$b------------------------
        Rscript _sh/exompeak.virus.R $MAIN $PEAKS $b _IN _RIP 0.025
	echo 'All done!'
done


In [30]:
cat _sh/exompeak.virus.R

args <- commandArgs(trailingOnly = TRUE)

print(args)
jobID  <- args[1]
OUTPUT <- args[2]
Sample <- args[3]
INPUT <- args[4]
IP <- args[5]
FDR <- args[6]

library(GenomicFeatures)
library(exomePeak)

GTF = '~/Projects/COVID19_m6A/virus/coronavirus_2_isolate_Wuhan-Hu-1.gff3'
######################################## read meta ######################################$
IP_BAM = paste(Sample, IP, '.bam', sep='')
INPUT_BAM = paste(Sample, INPUT, '.bam', sep='')

######################################## functions ######################################
txdb <- makeTxDbFromGFF(GTF, organism=NA )

setwd(jobID)
setwd("./virus_bam")

WINDOW = 50
STEP = 10
LENGTH = 200
ENRICH = 1
EXP = paste(Sample,'FDR', FDR,sep='_')

options(digits=5)
res <- exomepeak(
        TXDB = txdb,
        IP_BAM=IP_BAM,
        INPUT_BAM=INPUT_BAM,
        OUTPUT_DIR=paste('..',OUTPUT,sep='/'),
        EXPERIMENT_NAME=EXP,
        # options
        WINDOW_WIDTH = WINDOW,
        SLIDING_STEP = STEP,
        FRAGMENT_LENGTH

## 7. internal control

I copied the actual sequence for both _m6A+_ and _m6A-_ control fragments from the documentation of [N6-Methyladenosine Enrichment Kit](https://international.neb.com/-/media/nebus/files/manuals/manuale1610.pdf?rev=c064d5e232414f709ef8bccee56f7687&hash=AAA94B8FDA043DD5A610E14686552B5E0209EA94). Then, I manually made `bed` file and `fasta` file for each of these two sequences. Finally, I use `bowtie2` to map them to the `fastq` files which generated from reads that doesn't map to the human genome. 

Here, for a quality control of the samples, we are comparing:
$$\log_2\dfrac{RIP}{IN}$$
for both _m6A+_ and _m6A-_ controls with qPCR Ct. 

Let's see the sequences and `bed` files folowing with bash scripts I've used for alignment:

In [27]:
cat int_ctrl/m6A_neg.fa int_ctrl/m6A_pos.fa

>m6A Control RNA (Cypridina Luciferase): 1706 nt
GGAGACCCAAGCTTGGTACCGAGCTCGGATCCGCCACCATGAAGACCTTAATTCTTGCCGTTGCATTAGT
CTACTGCGCCACTGTTCATTGCCAGGACTGTCCTTACGAACCTGATCCACCAAACACAGTTCCAACTTCC
TGTGAAGCTAAAGAAGGAGAATGTATTGATAGCAGCTGTGGCACCTGCACGAGAGACATACTATCAGATG
GACTGTGTGAAAATAAACCAGGAAAAACATGTTGCCGAATGTGTCAGTATGTAATTGAATGCAGAGTAGA
GGCCGCAGGATGGTTTAGAACATTCTATGGAAAGAGATTCCAGTTCCAGGAACCTGGTACATACGTGTTG
GGTCAAGGAACCAAGGGCGGCGACTGGAAGGTGTCCATCACCCTGGAGAACCTGGATGGAACCAAGGGGG
CTGTGCTGACCAAGACAAGACTGGAAGTGGCTGGAGACATCATTGACATCGCTCAAGCTACTGAGAATCC
CATCACTGTAAACGGTGGAGCTGACCCTATCATCGCCAACCCGTACACCATCGGCGAGGTCACCATCGCT
GTTGTTGAGATGCCAGGCTTCAACATCACCGTCATTGAGTTCTTCAAACTGATCGTGATCGACATCCTCG
GAGGAAGATCTGTAAGAATCGCCCCAGACACAGCAAACAAAGGAATGATCTCTGGCCTCTGTGGAGATCT
TAAAATGATGGAAGATACAGACTTCACTTCAGATCCAGAACAACTCGCTAATCAGCCTAAGATCAACCAG
GAGTTTGACGGTTGTCCACTCTATGGAAATCCTGATGACGTTGCATACTGCAAAGGTCTTCTGGAGCCGT
ACAAGGACAGCTGCCGCAACCCCATCAACTTCTACTACTACACCATCTCCTGCGCCTTCGCCCGCTGTAT
GGGTGGAGACGAGCGAGCCTCACACGTG

In [30]:
cat second_dataset/_sh/int_ctrl.sh

echo '-------m6A pos-------'
bowtie2-build int_ctrl/m6A_pos.fa int_ctrl/m6A_pos

mkdir -p int_ctrl/
mkdir -p int_ctrl/virus/
mkdir -p int_ctrl/virus/m6A_pos
mkdir -p int_ctrl/virus/m6A_neg


for f in virus_fastq/*mate1;
    do f=`basename $f`;
    f2=${f/mate1/mate2};
    o=${f/_S*Unmapped.out.mate1/.bam};
	bowtie2 -p 12 --sensitive \
        -N 1 -x int_ctrl/m6A_pos \
        --no-unal \
        -1 virus_fastq/$f -2 virus_fastq/$f2 | samtools sort -o int_ctrl/virus/m6A_pos/$o;
done

echo '-------m6A neg--------'
bowtie2-build int_ctrl/m6A_neg.fa int_ctrl/m6A_neg

for f in virus_fastq/*mate1;
    do f=`basename $f`;
    f2=${f/mate1/mate2};
    o=${f/_S*Unmapped.out.mate1/.bam};
	bowtie2 -p 12 --sensitive \
        -N 1 -x int_ctrl/m6A_neg \
        --no-unal \
        -1 virus_fastq/$f -2 virus_fastq/$f2 | samtools sort -o int_ctrl/virus/m6A_neg/$o;
done

echo 'All done!'

In [31]:
cat int_ctrl/m6A_neg.bed int_ctrl/m6A_pos.bed

Unmodified	0	1706	neg	0	+
m6A	0	603	pos	0	+


Ok! Let's do some `log`:

In [6]:
def bam2count(bam,bed):
    cmd = 'bamToBed -i ' + bam + ' | intersectBed -s -wo -a - -b ' + bed + ' | cut -f10 | sort | uniq -c | awk \'{print $1}\''
    res = os.popen(cmd).read().split('\n')[0]
    return int(res)

def ctrl_count(sample):
    # path to ctrl bam files 
    IN_pos= 'int_ctrl/virus/m6A_pos/' + sample + '_IN.bam'
    RIP_pos='int_ctrl/virus/m6A_pos/' + sample + '_RIP.bam'
    IN_neg= 'int_ctrl/virus/m6A_neg/' + sample + '_IN.bam'
    RIP_neg='int_ctrl/virus/m6A_neg/' + sample + '_RIP.bam'
    # bed format 
    bed_pos = 'int_ctrl/m6A_pos.bed'
    bed_neg = 'int_ctrl/m6A_neg.bed'

    pos = [bam2count (bam, bed_pos) for bam in [IN_pos, RIP_pos]]
    neg = [bam2count (bam, bed_neg) for bam in [IN_neg, RIP_neg]]
    results = pos + neg
    return results

qPCR = pd.read_csv('RTqPCR.txt', sep='\t',index_col=0)

res = pd.DataFrame(
    data=[ctrl_count(S) for S in Samples], 
    index=Samples, columns=['IN_pos', 'RIP_pos', 'IN_neg', 'RIP_neg']
)

In [7]:
# log2(RIP / IN)
pos  = np.log2(res['RIP_pos'] / res['IN_pos'])
neg  = np.log2(res['RIP_neg'] / res['IN_neg'])

qPCR['m6A_pos_ctrl'] = pos 
qPCR['m6A_neg_ctrl'] = neg 
samples_with_low_Ct = qPCR[qPCR.iloc[:,0] <= 18].index.tolist()
qPCR.index = renameSamples

qPCR.to_csv('Results/qPCR_int_ctrl_log2RIPvsIN.txt')
qPCR

,CoV-2 Ct,m6A_pos_ctrl,m6A_neg_ctrl
COV00008,19.600,3.683781,-2.090802
COV00009,18.500,6.804162,-0.219748
COV00014,16.300,2.141356,-3.560361
COV00017,24.000,1.651047,-7.253435
COV00025,22.100,6.203816,-1.010379
COV00026,15.600,8.585372,2.581687
COV00030,17.000,8.069236,1.410737
COV00042,28.000,4.671282,-0.940368
COV00057,26.700,8.344047,-1.969256
COV00085,14.600,6.607715,0.843608


## 9. Count peak features 

The logic of this part is: 
1. Merge `exomepeak` results from all samples
2. Take intersect of each samples (`bed12`) 
3. Write it into fresh `bed6` format
4. Rename peaks from one transcript into uniq names

In [18]:
%%bash
# merge beds from all samples 
cat exomepeak/virus/*/peak.bed | grep -v '^ *#' | bed12ToBed6 | sort -k1,1 -k2,2n | \
# merge all peaks 
mergeBed -i - -c 4 -o distinct | \
# get intersects of all samples in the merged bed file
bedtools intersect -wo -F 0.85 -a - -b exomepeak/virus/*/peak.bed | \
awk '{print $6"\t"$7"\t"$8"\t"$9"\t"".""\t"$11}' | uniq | \
# rename peaks with unique names 
awk -F "\t" '{OFS=FS}{$4=$4"_peak"}; cnt[$4]++{$4=$4"_"cnt[$4]} 1' > merge_peaks.bed

count peak coverage in each sample (`bam` file) for downstream analysis:

In [20]:
%%bash
# peak count
for f in virus_bam/*bam; do
    base=`basename $f`;
    out=${base/.bam/.fc};
    bamToBed -i $f | intersectBed -split -c -a merge_peaks.bed -b - | \
    awk '{ print $4 "\t" $7}' > virus_count/$out; 
done

## 10. Motif analysis 
Extract peak sequences and search for `DRACH`, `RGAC`, `AAGAA` motifs.

In [ ]:
%%bash 
# get peak sequences
bedtools getfasta -name -s -fi virus/coronavirus_2_isolate_Wuhan-Hu-1.fasta -bed \
merge_peaks.bed -split -fo merge_peaks.fa

In [8]:
DRACH = re.compile('[AGT][AG]AC[ACT]') 
RGAC = re.compile('[AG]GAC') 
AAGAA = re.compile('AAGAA') 

def read_fasta(path):
    file = open(path)
    lines = file.read().splitlines()
    ids = [s[1:] for s in lines if '>' in s]
    n = [i for i,s in enumerate(lines) if '>' in s]
    n.append(len(lines))
    sequences = [''.join(lines[i+1:j]) for i,j in zip(n[:-1],n[1:])]
    file.close()
    fa = dict(zip(ids, sequences))
    return fa

def find_motifs(sam):
    motifs = [[f.split('::')[0], 
         len(DRACH.findall(sam[f])), 
         len(RGAC.findall(sam[f])), 
         len(AAGAA.findall(sam[f])),
         ','.join(DRACH.findall(sam[f]) + RGAC.findall(sam[f]) + AAGAA.findall(sam[f]))]
        for n, f in enumerate(sam)
    ]
    df = pd.DataFrame(motifs, columns=["loci","DRACH", "RGAC", "AAGAA","Sequence"])
    return df

peak_sequence = read_fasta('merge_peaks.fa')
motif_df = find_motifs(peak_sequence)
motif_df.replace(motif_df.iloc[1,4], np.nan, inplace=True)

motif_df.dropna(subset=["Sequence"], inplace=True)

motif_df.to_csv('Results/motif_analysis.txt', sep='\t')
motif_df

,loci,DRACH,RGAC,AAGAA,Sequence
0,S_peak,12,3,0,"GGACT,GAACC,AGACT,TGACC,AAACA,AAACT,TAACT,AAAC..."
2,S_peak_3,3,1,0,"GAACT,TGACA,TGACC,AGAC"
3,S_peak_4,23,7,1,"GAACT,TGACA,GGACT,GGACC,TAACC,TAACA,AGACC,AAAC..."
4,S_peak_5,2,0,0,"TAACA,AAACC"
5,S_peak_6,4,1,0,"GAACT,GGACC,AAACA,TAACA,GGAC"
...,...,...,...,...,...
163,N_peak_43,35,12,5,"GGACC,GGACC,TAACC,AAACA,AGACC,GGACA,TAACA,TGAC..."
164,N_peak_44,7,4,0,"GGACC,TAACC,AAACA,AGACC,GGACA,TAACA,TGACC,GGAC..."
165,N_peak_45,1,0,0,GAACA
166,N_peak_46,5,1,0,"AGACA,GAACT,AAACA,TGACC,TGACA,AGAC"


Let's filter some peaks with `DRACH` and `RGAC` motifs for below analysis:

In [9]:
peaks_with_motifs = motif_df[
    (motif_df.DRACH > 1) # & (motif_df.DRACH < 8) 
    &
    (motif_df.RGAC > 0)
    &
    (motif_df.AAGAA > 0)
].loci.tolist()
len(peaks_with_motifs)

48

## 11. m6A/input test of significance

We have discussed two scenarios with Honi to evaluate p-values for every peak in each samples (I'm following second option):
### 1st
- In IP samples, count reads mapping to the peak, and all peaks mapping to non-peak regions (i.e. background)
- In input, count the same, i.e. reads in the peak and non-peak
- that gives you a 2x2 contingency table, then calculate a p-value using Fisher's exact test

 _'non-peak' = it is total, reads mapping to all peaks_

### 2ed
- [ctest](https://github.com/lzcyzm/exomePeak/blob/master/R/ctest.R) function from exomepeak. [Here](https://github.com/lzcyzm/exomePeak/blob/master/R/peak.calling.module.R) is how that function is used

> `PEAK$loci2peak_merged=.get.peak.from.loci(READS_COUNT,ID,PARAMETERS)`

In [2]:
def read_counts(samples):
    '''
    Read raw counnt for IN and RIP into a pandas DataFrame    
    '''
    all_samples = np.concatenate([[N+'_IN',N+'_RIP'] for N in samples]).tolist()
    # Read count files as dataframe
    df_list = [
        pd.read_csv('virus_count/' + file, header=None, sep='\t', index_col=0) 
        for file in [N+'.fc' for N in all_samples]
    ]
    # concatenate them together 
    data = pd.concat(df_list, axis=1)
    data.columns = all_samples
    return data


def peak_names(samples):
    '''
    Extract peak names    
    '''
    peak_names = read_counts(samples).index.tolist()
    return peak_names


def add_border(plot,sig):
    '''
    For a given heatmap plot [plot], this function draw rectangles 
    around an array defined as list in the input indices.  
    '''
    i,j = sig
    plot.add_patch(Rectangle((i, j), 1, 1, fill=False, edgecolor='red', lw=5))

    
# https://towardsdatascience.com/how-to-create-a-plotly-visualization-and-embed-it-on-websites-517c1a78568b
def draw_heatmap(data, borders=None, save=False, name_it='', vmin=None,vmax=None):
    '''
    Draw/save heatmap plot with option to draw rectangle around given cells 
    '''
    heat_map = sns.heatmap(data,linewidth=2, vmin=vmin,vmax=vmax)
    # add borders 
    if borders: 
        for sig in borders: add_border (heat_map,sig)
    bottom, top = heat_map.get_ylim()
    heat_map.set_ylim(bottom + 0.5, top - 0.5)
    heat_map.set_yticklabels(heat_map.get_yticklabels(), rotation=0)
    if save:
        plt.savefig(name_it + '.png')
        plt.savefig(name_it + '.pdf')
    else:
        plt.show()

Peak_names = peak_names(Samples)
Counts = read_counts(Samples)

Counts.to_csv('Results/raw_counts.txt', sep='\t')

Let's do some `R`! 

In [17]:
%reload_ext rpy2.ipython

In [18]:
%%R 
# These function implemented from exomePeak R package. 

ctest <-  function(IP,INPUT,TOTAL_IP,TOTAL_INPUT,FOLD=1,minimal_counts_in_fdr = 10) {
  
  # input check
  if (length(IP) != length(INPUT)) { 
      stop("The IP and INPUT of ctest must be of the same length.", call. = TRUE, domain = NULL) 
  }
  # replace 0 with 1
  IP=pmax(IP,1)
  INPUT=pmax(INPUT,1)
  
  # calculate p
  a=TOTAL_IP*FOLD
  b=TOTAL_INPUT
  p=a/(a+b)
  
  # get total observation
  total=IP+INPUT
  
  # cdf
  log.p=pbinom(IP-1, total, p, lower.tail = FALSE, log.p = TRUE)
  
  # calculate p  
  pvalues=exp(log.p)
  
  # calculate fdr
  log.fdr=log(p.adjust(pvalues, method = "fdr"))
  
  # with significant number of reads only
  ID=which( (IP+INPUT) > minimal_counts_in_fdr)
  log.fdr_sig=log(p.adjust(pvalues[ID], method = "fdr"))
  log.fdr[ID]=log.fdr_sig
  
  # fold enrichment
  log.fc=log((pmax(1,IP)/sum(IP))/(pmax(1,INPUT)/sum(INPUT)))
  
  # output result
  PW=list(log.p=log.p,log.fdr=log.fdr,log.fc=log.fc)
  return(PW)

}

peak.calling <- function(READS_COUNT,SAMPLE_ID){
    
    # check points comparison
    IP=READS_COUNT[,paste(SAMPLE_ID,'RIP',sep='_')]
    INPUT=READS_COUNT[,paste(SAMPLE_ID,'IN',sep='_')]
    
    PW = ctest(IP,INPUT,sum(IP),sum(INPUT))
    
    # output peaks
    return(PW)
}

In [19]:
%%R -i Counts,Samples,Peak_names -o RESULTS
Samples = unlist(Samples)
Peak_names = unlist(Peak_names)

RESULTS = list()
for (S in Samples){RESULTS[[S]] = peak.calling(Counts,S)}

`ctest` results for every peaks in every samples are ready to go! 

In [20]:
log_p  = pd.DataFrame([np.array(R[0]) for R in RESULTS], index=Samples,columns=Peak_names)
log_fdr= pd.DataFrame([np.array(R[1]) for R in RESULTS], index=Samples,columns=Peak_names)
log_fc = pd.DataFrame([np.array(R[2]) for R in RESULTS], index=Samples,columns=Peak_names)

In [21]:
list(ob.r.names(RESULTS)) == Samples

True

## 12. Mutation analysis 

https://www.datacamp.com/community/tutorials/fuzzy-string-python

Levenshtein Distance Equation
https://medium.com/@ethannam/understanding-the-levenshtein-distance-equation-for-beginners-c4285a5604f0

In [237]:
# peak sequences as dataframe
peak_seq_df = pd.DataFrame(
    list(peak_sequence.items()),
    columns = ['coordinate','sequence']
) 
peak_seq_df.index = [peak_seq_df.loc[p,'coordinate'].split('::')[0] for p in peak_seq_df.index]

In [238]:
# WGS sequencing of patient samples as datafram
files = glob.glob('virus_WGS/*.fasta')
sample_seq_df = pd.DataFrame(columns = ['sequence']) 
for f in files:
    sample = f.replace('virus_WGS/NM-n','').replace('.fasta','')
    sample_seq_df.loc[sample] = list(read_fasta(f).values())

In [243]:
# build empty numpy array 
mutation_mat = np.zeros((
    sample_seq_df.shape[0], 
    peak_seq_df.shape[0]  
))

# nested for loop to run string matching between peak and sample sequencing
for i, sample in enumerate(sample_seq_df['sequence'].to_list() ):
    for j,peak in enumerate(peak_seq_df['sequence'].to_list() ):
        mutation_mat[i,j] = fuzz.partial_ratio(sample, peak)
        
# column normalization of partial_ratio score
mutation_mat_norm = pp.minmax_scale(mutation_mat, axis=0)

mutation_df = pd.DataFrame(
    mutation_mat,
    index=sample_seq_df.index.to_list(), columns=peak_seq_df.index.to_list()
)

mutation_norm_df = pd.DataFrame(
    mutation_mat_norm,
    index=sample_seq_df.index.to_list(), columns=peak_seq_df.index.to_list()
)

mutation_df.to_csv('Results/mutation_mat_fuzzy_partial_ratio.txt', sep='\t')
mutation_norm_df.to_csv('Results/mutation_mat_norm.txt', sep='\t')

In [396]:
# fresh = (100 - mutation_mat) - 30 
# fresh_df = pd.DataFrame(
#     fresh,
#     index=sample_seq_df.index.to_list(), columns=peak_seq_df.index.to_list()
# )

In [452]:
# mutated_peaks = mutation_norm_df.sum(axis=0) > 0

In [472]:
# plt.figure(figsize=(8,5))
# draw_heatmap(mutation_norm_df.loc[:,mutated_peaks], )
# mutation_norm_df.loc[:,mutated_peaks]

(not used scripts)

### Total number of reads:
1. Using `samtools` to convert `bam` files to `fastq` files 
2. Count total number of lines / by 4 as total number of reads 

virus

In [490]:
%%bash --err error
mkdir -p total
echo 'sample, reads' > total/virus.txt
for f in virus_bam/*bam;
	do 
    name=`basename $f`
    name=${name/.bam/}
    declare -i a=`samtools fastq $f | wc -l`
    read=$((a / 4))
    echo $name,$read >> total/virus.txt
done

human 

In [491]:
%%bash --err error
echo 'sample, reads' > total/human.txt
for f in */bam/*.bam; 
	do 
    name=`basename $f`
    name=${name/.bam/}
    declare -i a=`samtools fastq $f | wc -l`
    read=$((a / 4))
    echo $name,$read >> total/human.txt
done

ctrl m6A_neg

In [16]:
%%bash --err error
echo 'sample, reads' > total/m6A_neg.txt
for f in */int_ctrl/virus/m6A_neg/*bam; 
	do 
    name=`basename $f`
    name=${name/_S[0-9]*/}
    name=${name/.bam/}
    declare -i a=`samtools fastq $f | wc -l`
    read=$((a / 4))
    echo $name,$read >> total/m6A_neg.txt
done

ctrl m6A_pos

In [15]:
%%bash --err error
echo 'sample, reads' > total/m6A_pos.txt
for f in */int_ctrl/virus/m6A_pos/*bam; 
	do 
    name=`basename $f`
    name=${name/_S[0-9]*/}
    name=${name/.bam/}
    declare -i a=`samtools fastq $f | wc -l`
    read=$((a / 4))
    echo $name,$read >> total/m6A_pos.txt
done

In [266]:
aligned = ['human','virus','m6A_pos','m6A_neg']
total_df = pd.concat([pd.read_csv('total/'+a+'.txt', sep=',', index_col=0).sort_index() for a in aligned],axis=1)
total_df.columns = aligned

total_df.to_csv('total_reads.txt', sep='\t')

#### Read and normalize counts 
Log normalization of peak counts 

$$\log_2 \dfrac{
    \dfrac{\text{m6A counts in the peak}}{\text{m6A total counts}}
    }{
    \dfrac{\text{input counts in the peak}}{\text{input total counts}}
    }
    $$


In [ ]:
# def norm_counts(counts, samples, total_reads, norm='l1'):
#     all_samples = np.concatenate([[N+'_IN',N+'_RIP'] for N in samples]).tolist()    
#     peak_names = counts.index.tolist()
#     n_sam = len(all_samples)
#     # normalize
#     total = total_reads.loc[all_samples,].sum(axis=1)
#     total = np.array(total).T
#     data = np.array(counts[all_samples]) 
#     # log2 (RIP in peak / RIP total) / (IN in peak / IN total)
#     res = np.log2(
#         (
#             # RIP / RIP ctrl
#             data [:,np.array(range(1,n_sam,2))] 
#             /
#             total[np.array(range(1,n_sam,2))]
#         ) / (
#             # IN / IN ctrl
#             data [:,np.array(range(0,n_sam,2))] 
#             /
#             total[np.array(range(0,n_sam,2))]
#         )
#     )
#     # normalization by samples
#     if norm == None:
#         res = res
#     elif norm == 'min':
#         res = pp.minmax_scale(res, feature_range=(0, 1), axis=1)
#     elif norm == 'zscore':
#         res = stats.zscore(res, axis=0, ddof=1)
#     else:
#         res = pp.normalize(res, axis=1, norm=norm)
#     res_df = pd.DataFrame(data=res, columns=samples, index=peak_names)
#     return res_df


# Total = pd.read_csv('total_reads.txt', sep = '\t', index_col=0)  
# Norm_counts = norm_counts(Counts, Samples, Total, norm=None)
# Norm_counts.to_csv('norm_counts.txt', sep='\t')